Importación de las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

**DATOS DE CABEZERA**

In [2]:
data = {
    'clay': [20.5],
    'depth': [90.0],
    'iom': [2.25],
    'nsteps': [132]
}

df_head = pd.DataFrame(data)
print(df_head)

   clay  depth   iom  nsteps
0  20.5   90.0  2.25     132


**IMPORTACIÓN DF DE ENTRADA**

In [3]:
df = pd.read_csv('simulatedData_intensive.csv')
df.dtypes

year         int64
month        int64
modern     float64
Tmp        float64
Rain       float64
Evap       float64
C_inp      float64
FYM          int64
PC           int64
DPM_RPM    float64
dtype: object

**Definición de funciones**

In [4]:
def RMF_Tmp (TEMP):
         
    if(TEMP<-5.0): 
        RM_TMP=0.0
    else:
        RM_TMP=47.91/(np.exp(106.06/(TEMP+18.27))+1.0)
   
    return RM_TMP


def RMF_Moist (RAIN, PEVAP, clay, depth, PC, SWC):
        
    RMFMax = 1.0
    RMFMin = 0.2

    SMDMax=-(20+1.3*clay-0.01*(clay*clay)) 
    SMDMaxAdj = SMDMax * depth / 23.0 
    
    SMD1bar = 0.444 * SMDMaxAdj
    
    DF = RAIN - 0.75 * PEVAP 
    minSWCDF= np.min(np.array([0.0, SWC[0]+DF]))
    
    SMDBare = 0.556 * SMDMaxAdj 
    minSMDBareSWC= np.min (np.array([SMDBare, SWC[0]])) 
     
    if(PC==1):
        SWC[0] = np.max(np.array([SMDMaxAdj, minSWCDF]))
    else:
        SWC[0] = np.max(np.array([minSMDBareSWC, minSWCDF]))
       
    if(SWC[0]>SMD1bar): 
        RM_Moist = 1.0
    else:
        RM_Moist = (RMFMin + (RMFMax - RMFMin) * (SMDMaxAdj - SWC[0]) / (SMDMaxAdj - SMD1bar) )   
    
    return RM_Moist


def RMF_PC (PC): 
    if (PC==0): 
        RM_PC = 1.0
    else: 
        RM_PC = 0.6

    return RM_PC



def decomp(timeFact, DPM,RPM,BIO,HUM, IOM, SOC, DPM_Rage, RPM_Rage, \
           BIO_Rage, HUM_Rage, IOM_Rage, Total_Rage, modernC, RateM, clay, C_Inp, FYM_Inp, DPM_RPM):

    zero = 0e-8
    DPM_k = 10.0
    RPM_k = 0.3
    BIO_k = 0.66
    HUM_k = 0.02 
    
    conr = np.log(2.0) / 5568.0

    tstep = 1.0/timeFact    
      
    exc = np.exp(-conr*tstep) 
      
    DPM1 = DPM[0] * np.exp(-RateM*DPM_k*tstep)
    RPM1 = RPM[0] * np.exp(-RateM*RPM_k*tstep)      
    BIO1 = BIO[0] * np.exp(-RateM*BIO_k*tstep)      
    HUM1 = HUM[0] * np.exp(-RateM*HUM_k*tstep) 
      
    DPM_d = DPM[0] - DPM1
    RPM_d = RPM[0] - RPM1      
    BIO_d = BIO[0] - BIO1
    HUM_d = HUM[0] - HUM1 
      
    x=1.67*(1.85+1.60*np.exp(-0.0786*clay))
                    
    DPM_co2 = DPM_d * (x / (x+1))
    DPM_BIO = DPM_d * (0.46 / (x+1))
    DPM_HUM = DPM_d * (0.54 / (x+1))
      
    RPM_co2 = RPM_d * (x / (x+1))
    RPM_BIO = RPM_d * (0.46 / (x+1))
    RPM_HUM = RPM_d * (0.54 / (x+1))    
      
    BIO_co2 = BIO_d * (x / (x+1))
    BIO_BIO = BIO_d* (0.46 / (x+1))
    BIO_HUM = BIO_d * (0.54 / (x+1))
      
    HUM_co2 = HUM_d * (x / (x+1))
    HUM_BIO = HUM_d * (0.46 / (x+1))
    HUM_HUM = HUM_d * (0.54 / (x+1))  
              
    DPM[0] = DPM1
    RPM[0] = RPM1
    BIO[0] = BIO1 + DPM_BIO + RPM_BIO + BIO_BIO + HUM_BIO
    HUM[0] = HUM1 + DPM_HUM + RPM_HUM + BIO_HUM + HUM_HUM    
      
    PI_C_DPM = DPM_RPM / (DPM_RPM + 1.0) * C_Inp
    PI_C_RPM =     1.0 / (DPM_RPM + 1.0) * C_Inp

    FYM_C_DPM = 0.49*FYM_Inp
    FYM_C_RPM = 0.49*FYM_Inp      
    FYM_C_HUM = 0.02*FYM_Inp   
      
    DPM[0] = DPM[0] + PI_C_DPM + FYM_C_DPM
    RPM[0] = RPM[0] + PI_C_RPM + FYM_C_RPM  
    HUM[0] = HUM[0] + FYM_C_HUM
      
    DPM_Ract = DPM1 *np.exp(-conr*DPM_Rage[0])
    RPM_Ract = RPM1 *np.exp(-conr*RPM_Rage[0]) 
    BIO_Ract = BIO1 *np.exp(-conr*BIO_Rage[0])
    HUM_Ract = HUM1 *np.exp(-conr*HUM_Rage[0])   

    DPM_BIO_Ract = DPM_BIO * np.exp(-conr*DPM_Rage[0])
    RPM_BIO_Ract = RPM_BIO * np.exp(-conr*RPM_Rage[0])
    BIO_BIO_Ract = BIO_BIO * np.exp(-conr*BIO_Rage[0])
    HUM_BIO_Ract = HUM_BIO * np.exp(-conr*HUM_Rage[0])
      
    DPM_HUM_Ract = DPM_HUM * np.exp(-conr*DPM_Rage[0])
    RPM_HUM_Ract = RPM_HUM * np.exp(-conr*RPM_Rage[0])
    BIO_HUM_Ract = BIO_HUM * np.exp(-conr*BIO_Rage[0])
    HUM_HUM_Ract = HUM_HUM * np.exp(-conr*HUM_Rage[0])
      
    IOM_Ract = IOM[0] *np.exp(-conr*IOM_Rage[0]) 
      
    PI_DPM_Ract = modernC * PI_C_DPM
    PI_RPM_Ract = modernC * PI_C_RPM
      
    FYM_DPM_Ract = modernC * FYM_C_DPM
    FYM_RPM_Ract = modernC * FYM_C_RPM
    FYM_HUM_Ract = modernC * FYM_C_HUM          
      
    DPM_Ract_new = FYM_DPM_Ract + PI_DPM_Ract + DPM_Ract*exc
    RPM_Ract_new = FYM_RPM_Ract + PI_RPM_Ract + RPM_Ract*exc    
    
    BIO_Ract_new = (BIO_Ract + DPM_BIO_Ract + RPM_BIO_Ract + 
                    BIO_BIO_Ract + HUM_BIO_Ract )*exc
      
    HUM_Ract_new = FYM_HUM_Ract + (HUM_Ract + DPM_HUM_Ract + 
                                   RPM_HUM_Ract + BIO_HUM_Ract + HUM_HUM_Ract )*exc  
      
    SOC[0] = DPM[0] + RPM[0] + BIO[0] + HUM[0] + IOM[0]     
      
    Total_Ract = DPM_Ract_new + RPM_Ract_new + BIO_Ract_new + HUM_Ract_new + IOM_Ract
      
    if(DPM[0]<=zero): 
        DPM_Rage[0] = zero
    else:
        DPM_Rage[0] = ( np.log(DPM[0]/DPM_Ract_new) ) / conr

    
    if(RPM[0]<=zero):
        RPM_Rage[0] = zero
    else:
        RPM_Rage[0] = (np.log(RPM/RPM_Ract_new) ) / conr 
    
    if(BIO[0]<=zero):
        BIO_Rage[0] = zero
    else:
        BIO_Rage[0] = ( np.log(BIO/BIO_Ract_new) ) / conr
    
    
    if(HUM[0]<=zero):
        HUM_Rage[0] = zero
    else:
        HUM_Rage[0] = ( np.log(HUM/HUM_Ract_new) ) / conr
    
    
    if(SOC[0]<=zero):
        Total_Rage[0] = zero
    else:
        Total_Rage[0] = ( np.log(SOC[0]/Total_Ract) ) / conr    
    return


def RothC(timeFact, DPM,RPM,BIO,HUM,IOM, SOC, DPM_Rage, RPM_Rage, BIO_Rage, HUM_Rage, IOM_Rage,\
                Total_Rage, modernC, clay, depth,TEMP,RAIN,PEVAP,PC,DPM_RPM, \
                C_Inp, FYM_Inp, SWC): 
    RM_TMP = RMF_Tmp(TEMP)
    RM_Moist = RMF_Moist(RAIN, PEVAP, clay, depth, PC, SWC)
    RM_PC = RMF_PC(PC)

    RateM = RM_TMP*RM_Moist*RM_PC
      
    decomp(timeFact, DPM,RPM,BIO,HUM, IOM, SOC, DPM_Rage, RPM_Rage, BIO_Rage, HUM_Rage, IOM_Rage, Total_Rage, modernC, RateM, clay, C_Inp, FYM_Inp, DPM_RPM)
    return

In [5]:
df.columns =['t_year', 't_month', 't_mod', 't_tmp','t_rain','t_evap', 't_C_Inp', 't_FYM_Inp', 't_PC', 't_DPM_RPM']

In [6]:
# set initial  values   
DPM = [2.0]  
RPM = [3.0]  
BIO = [1.0]  
HUM = [5.0]  
SOC = [28.5]  

DPM_Rage = [10.0]  
RPM_Rage = [15.0]  
BIO_Rage = [5.0] 
HUM_Rage = [20.0]  
IOM_Rage = [50000.0]  

# set initial soil water content (deficit)
SWC = [100.0]  
TOC1 = 28.5  
   

In [7]:
clay = df_head.loc[0,"clay"]
depth = df_head.loc[0,"depth"]
IOM = [df_head.loc[0,"iom"]]
nsteps = df_head.loc[0,"nsteps"]

In [8]:
k = -1
j = -1
      
SOC[0] = DPM[0]+RPM[0]+BIO[0]+HUM[0]+IOM[0]
print (j, DPM[0], RPM[0], BIO[0], HUM[0], IOM[0], SOC[0])

timeFact = 12
test = 100.0

-1 2.0 3.0 1.0 5.0 2.25 13.25


In [9]:
while (test > 1E-6):
    k = k + 1
    j = j + 1 
    if( k == timeFact):
        k = 0 
    TEMP = df.t_tmp[k]
    RAIN = df.t_rain[k]
    PEVAP =df.t_evap[k]   
    PC = df.t_PC[k]
    DPM_RPM = df.t_DPM_RPM[k] 
    C_Inp = df.t_C_Inp[k]
    FYM_Inp = df.t_FYM_Inp[k]
  
    modernC = df.t_mod[k] / 100.0   
    Total_Rage=[0.0]
    RothC(timeFact, DPM,RPM,BIO,HUM,IOM, SOC, DPM_Rage, RPM_Rage, BIO_Rage, HUM_Rage, IOM_Rage, Total_Rage, \
        modernC, clay, depth,TEMP,RAIN,PEVAP,PC,DPM_RPM, C_Inp, FYM_Inp, SWC)  
    
    #each a year calculates the difference between previous year and current year (counter =12 monthly model)
    if (np.mod(k+1, timeFact)== 0):
        TOC0 = TOC1
        TOC1 =DPM[0]+RPM[0]+BIO[0]+HUM[0]
        test = abs(TOC1-TOC0)   


In [10]:
Total_Delta = (np.exp(-Total_Rage[0]/8035.0) - 1.0) * 1000.0     

#Estado final cuando se ha alcanzado el equilibrio 
print(
    f"j (índice de tiempo): {j}\n"
    f"DPM[0] (Descomposición de Material Vegetal Descomponible): {DPM[0]}\n"
    f"RPM[0] (Descomposición de Material Vegetal Resistente): {RPM[0]}\n"
    f"BIO[0] (Biomasa Microbiana): {BIO[0]}\n"
    f"HUM[0] (Materia Orgánica Humificada): {HUM[0]}\n"
    f"IOM[0] (Materia Orgánica Inerte): {IOM[0]}\n"
    f"SOC[0] (Carbono Orgánico del Suelo): {SOC[0]}\n"
    f"Total_Delta (Cambio Total en la Actividad del Radiocarbono): {Total_Delta}\n"
    f"Diferencia final: {test}"
    
)



j (índice de tiempo): 6887
DPM[0] (Descomposición de Material Vegetal Descomponible): 0.28180321682708304
RPM[0] (Descomposición de Material Vegetal Resistente): 4.060117811075822
BIO[0] (Biomasa Microbiana): 0.47975115775368815
HUM[0] (Materia Orgánica Humificada): 18.258991652266545
IOM[0] (Materia Orgánica Inerte): 2.25
SOC[0] (Carbono Orgánico del Suelo): 25.33066383792314
Total_Delta (Cambio Total en la Actividad del Radiocarbono): [-125.80505324]
Diferencia final: 9.920159342868828e-07


In [11]:
year_list = [[1, j+1, DPM[0], RPM[0], BIO[0], HUM[0], IOM[0], SOC[0], Total_Delta[0]]]

month_list = []        

for  i in range(timeFact, nsteps):
  
    TEMP = df.t_tmp[i]
    RAIN = df.t_rain[i]
    PEVAP =df.t_evap[i]
    
    PC = df.t_PC[i]
    DPM_RPM = df.t_DPM_RPM[i]
    
    C_Inp = df.t_C_Inp[i]
    FYM_Inp = df.t_FYM_Inp[i]
    
    modernC = df.t_mod[i] / 100.0
    
    RothC(timeFact, DPM, RPM, BIO, HUM, IOM, SOC, DPM_Rage, RPM_Rage, BIO_Rage, HUM_Rage, IOM_Rage, Total_Rage, \
      modernC, clay, depth, TEMP, RAIN, PEVAP, PC, DPM_RPM, C_Inp, FYM_Inp, SWC)
         
    Total_Delta = (np.exp(-Total_Rage[0]/8035.0) - 1.0) * 1000.0
    
    #print(C_Inp, FYM_Inp, TEMP, RAIN, PEVAP, SWC[0],  PC,  DPM[0],RPM[0],BIO[0],HUM[0], IOM[0], SOC[0])
    
    month_list.insert(i-timeFact, [df.loc[i,"t_year"],df.loc[i,"t_month"], DPM[0],RPM[0],BIO[0],HUM[0], IOM[0], SOC[0], Total_Delta[0]])
        
    if(df.t_month[i] == timeFact):
        timeFact_index = int(i/timeFact) 
        year_list.insert(timeFact_index, [df.loc[i,"t_year"],df.loc[i,"t_month"], DPM[0],RPM[0],BIO[0],HUM[0], IOM[0], SOC[0], Total_Delta[0]])
        #print( i, DPM, RPM, BIO, HUM, IOM, SOC, Total_Delta)

Se guardan los resultados finales por mes y su respectivo resumen anual para un análisis mas general

In [12]:
output_years = pd.DataFrame(year_list, columns=["Year","Month","DPM_t_C_ha","RPM_t_C_ha","BIO_t_C_ha","HUM_t_C_ha","IOM_t_C_ha","SOC_t_C_ha","deltaC"])     
output_months = pd.DataFrame(month_list, columns=["Year","Month","DPM_t_C_ha","RPM_t_C_ha","BIO_t_C_ha","HUM_t_C_ha","IOM_t_C_ha","SOC_t_C_ha","deltaC"])

output_years.to_csv("year_results_intensive.csv", index = False)
output_months.to_csv("months_results_intensive.csv", index = False)

In [13]:
year = pd.read_csv("year_results_intensive.csv")
months = pd.read_csv("months_results_intensive.csv")
year = year.drop(df.index[0])

year

,Year,Month,DPM_t_C_ha,RPM_t_C_ha,BIO_t_C_ha,HUM_t_C_ha,IOM_t_C_ha,SOC_t_C_ha,deltaC
1,2011,12,0.215031,3.896198,0.468792,18.238315,2.25,25.068337,-126.749903
2,2012,12,0.240164,3.874153,0.465346,18.217494,2.25,25.047156,-126.827512
3,2013,12,0.263773,3.900842,0.465173,18.199771,2.25,25.079558,-126.840281
4,2014,12,0.235663,3.839035,0.460296,18.176935,2.25,24.961930,-127.380704
5,2015,12,0.247716,3.755577,0.448084,18.132938,2.25,24.834315,-127.929639
6,2016,12,0.234984,3.694979,0.444866,18.095641,2.25,24.720470,-128.417974
7,2017,12,0.230599,3.698178,0.447807,18.063832,2.25,24.690416,-128.646368
8,2018,12,0.203382,3.671197,0.447803,18.033628,2.25,24.606009,-129.099574
9,2019,12,0.248316,3.714471,0.446222,17.996505,2.25,24.655513,-129.095525
10,2020,12,0.229135,3.672764,0.444403,17.958960,2.25,24.555261,-129.615039


In [15]:
months

,Year,Month,DPM_t_C_ha,RPM_t_C_ha,BIO_t_C_ha,HUM_t_C_ha,IOM_t_C_ha,SOC_t_C_ha,deltaC
0,2011,1,0.257253,4.101475,0.484257,18.264341,2.25,25.357326,-125.670248
1,2011,2,0.218884,4.125385,0.488334,18.269295,2.25,25.351898,-125.660798
2,2011,3,0.241391,4.197456,0.490501,18.272179,2.25,25.451528,-125.291277
3,2011,4,0.288754,4.206476,0.493728,18.276332,2.25,25.515290,-125.054492
4,2011,5,0.291772,4.192108,0.499342,18.283345,2.25,25.516567,-125.049360
...,...,...,...,...,...,...,...,...,...
115,2020,8,0.151254,3.535599,0.453690,17.980439,2.25,24.370983,-130.111866
116,2020,9,0.132365,3.457324,0.442501,17.960553,2.25,24.242744,-130.617694
117,2020,10,0.207679,3.561347,0.440432,17.956222,2.25,24.415681,-130.039128
118,2020,11,0.196674,3.598679,0.442791,17.957712,2.25,24.445856,-129.962283
